<img src = "https://www.pucsp.br/sites/default/files/download/brasao-PUCSP-assinatura-principal-RGB.png" width="15%"/>

*Integrantes*

<table align="left">
    <tr>
        <td>Nome:</td>
        <td><b>Beatriz Gil</b></td>
        <td>RA:</td>
        <td><b>RA00297693</b></td>
    </tr>
    <tr>
        <td>Nome:</td>
        <td><b>Lucas Gregorio</b></td>
        <td>RA:</td>
        <td><b>RA00297813</b></td>
    </tr>
    <tr>
        <td>Nome:</td>
        <td><b>Vinícius Perez</b></td>
        <td>RA:</td>
        <td><b>RA00297669</b></td>
    </tr>
</table>

# Bibliotecas

In [1]:
from flask import request,Response,Flask
import os
import sqlite3
import pandas as pd

# Funções principais que retornam um DF de cada tabela do SQL

In [2]:
def Jogos():
    conn = sqlite3.connect('basquete.db')
    strSQL ="SELECT* from Jogos"
    df = pd.read_sql_query(strSQL, conn)
    conn.close()
    return df

def Teams():
    conn = sqlite3.connect('basquete.db')
    strSQL ="SELECT* from Teams"
    df = pd.read_sql_query(strSQL, conn)
    conn.close()
    return df

def Players():
    conn = sqlite3.connect('basquete.db')
    strSQL ="SELECT* from Players"
    df = pd.read_sql_query(strSQL, conn)
    conn.close()
    return df

def Info():
    conn = sqlite3.connect('basquete.db')
    strSQL ="SELECT* from Info"
    df = pd.read_sql_query(strSQL, conn)
    conn.close()
    return df

# Funções secundárias (pois não filtramos toda a tabela) que retornam um DF de cada tabela do SQL

In [3]:
#Filtrando por Jogadores com idade menor ou igual a 30 anos e ativos na temporada de 2022
def Age_Less_30():
    conn = sqlite3.connect('basquete.db')
    strSQL ="""SELECT Teams.abbreviation, Info.PLAYER, Info.Age, Info.SEASON, Info.POSITION
               from Info
               INNER JOIN Players on Players.PLAYER_ID = Info.idPlayer_IdFK
               INNER JOIN Teams on Teams.TeamID = Info.idTeamIDFK
               where Info.Age <=30 AND Players.is_active = 1
               order by Info.Age DESC; """
    df = pd.read_sql_query(strSQL, conn)
    return df

#Filtrando por Jogadores com altura superior a 2 metros
def Height_Greater_30():
    conn = sqlite3.connect('basquete.db')
    strSQL ="""SELECT Teams.abbreviation, Info.PLAYER, Info.HEIGHT, Info.SEASON, Info.POSITION
               from Info
               INNER JOIN Players on Players.PLAYER_ID = Info.idPlayer_IdFK
               INNER JOIN Teams on Teams.TeamID = Info.idTeamIDFK
               where Info.HEIGHT > 2 AND Players.is_active = 1
               order by Info.HEIGHT DESC; """
    df = pd.read_sql_query(strSQL, conn)
    return df

#Ordenando do maior pro menor, os times que tiveram a maior soma de pontos durante o torneio de 2022
def Sort_Sum_PTS():
    conn = sqlite3.connect('basquete.db')
    strSQL ="""SELECT Teams.TeamID, Teams.abbreviation, Teams.city, Teams.state, Teams.year_founded, SUM(Jogos.PTS) as sum_PTS
               from Teams
               INNER JOIN Jogos on Jogos.idTeamIDFK = Teams.TeamID
               group by Teams.abbreviation
               order by sum_PTS DESC; """
    df = pd.read_sql_query(strSQL, conn)
    return df

#Comparação da quatidade de Players Ativos e Inativos na Temporada de 2022
def Players_Active_NoActive():
    conn = sqlite3.connect('basquete.db')
    strSQL ="""SELECT COUNT(CASE when is_active = 1 THEN 1 END) AS QTD_Players_ATIVOS, 
                      COUNT(CASE when is_active = 0 THEN 1 END) AS QTD_Players_INATIVOS
               FROM Players; """
    df = pd.read_sql_query(strSQL, conn)
    return df

#Ordenando do maior pro menor, os times com a maior quantidade de vitórias em 2022 e a quantidade de derrotas
def Jogos_Win_Loss():
    conn = sqlite3.connect('basquete.db')
    strSQL ="""SELECT abbreviation, COUNT(CASE when Jogos.WL = 'W' THEN 1 END) as QTD_Wins,
                                    COUNT(CASE when Jogos.WL = 'L' THEN 1 END) as QTD_Loss
               from Teams
               INNER JOIN Jogos on Jogos.idTeamIDFK = Teams.TeamID
               group by Teams.abbreviation
               order by QTD_Wins DESC; """
    df = pd.read_sql_query(strSQL, conn)
    return df

#Qual jogador pertence a cada time
def Player_Teams():
    conn = sqlite3.connect('basquete.db')
    strSQL ="""SELECT Info.PLAYER, Teams.abbreviation, Teams.team_name 
               from Info
               INNER JOIN Teams on Info.idTeamIDFK = Teams.TeamID
               order by abbreviation ASC; """
    df = pd.read_sql_query(strSQL, conn)
    return df

# criar uma instância do objeto Flask

In [4]:
API = Flask(__name__)

# Retorna as funções criadas e retorna um Json (no caso a API)

In [5]:
#Capa
@API.route('/')
def home():
    saudacao = "<div style='display:flex; flex-direction:column; align-items:center;'>"
    saudacao += "<h1 style='font-size: 35px; font-family: 'Cooper Black', serif; color: black; text-align:center;'>BEM VINDOS A NOSSA API DA NBA!</h1>"
    saudacao += "<div style='display:flex; justify-content: space-between; align-items:center; width: 105%;'>"
    saudacao += "</a>"
    saudacao += "<img src='https://cdn.wallpapersafari.com/39/69/XfbzIN.png' alt='Imagem de fundo da NBA' style='width:100%;'>"
    saudacao += "</div>"
    saudacao += "</div>"
    return saudacao

#Tabela Jogos
@API.route("/Jogos",methods=['GET'])
def getJogos():
    df = Jogos()
    return Response(df.to_json(orient="records"), mimetype='application/json')

#Tabela Teams
@API.route("/Teams",methods=['GET'])
def getTeams():
    df = Teams()
    return Response(df.to_json(orient="records"), mimetype='application/json')

#Tabela Players
@API.route("/Players",methods=['GET'])
def getPlayers():
    df = Players()
    return Response(df.to_json(orient="records"), mimetype='application/json')

#Tabela Info
@API.route("/Info",methods=['GET'])
def getInfo():
    df = Info()
    return Response(df.to_json(orient="records"), mimetype='application/json')

#Tabela Age_Less_30
@API.route("/Age_Less_30",methods=['GET'])
def get_Age_Less_30():
    df = Age_Less_30()
    return Response(df.to_json(orient="records"), mimetype='application/json')

#Tabela Height_Greater_30
@API.route("/Height_Greater_30",methods=['GET'])
def get_Height_Greater_30():
    df = Height_Greater_30()
    return Response(df.to_json(orient="records"), mimetype='application/json')

#Tabela Sort_Sum_PTS
@API.route("/Sort_Sum_PTS",methods=['GET'])
def get_Sort_Sum_PTS():
    df = Sort_Sum_PTS()
    return Response(df.to_json(orient="records"), mimetype='application/json') 

#Players_Active_NoActive
@API.route("/Players_Active_NoActive",methods=['GET'])
def get_Players_Active_NoActive():
    df = Players_Active_NoActive()
    return Response(df.to_json(orient="records"), mimetype='application/json')

#Jogos_Win_Loss
@API.route("/Jogos_Win_Loss",methods=['GET'])
def get_Jogos_Win_Loss():
    df = Jogos_Win_Loss()
    return Response(df.to_json(orient="records"), mimetype='application/json')

#Jogos_Win_Loss
@API.route("/Player_Teams",methods=['GET'])
def get_Player_Teams():
    df = Player_Teams()
    return Response(df.to_json(orient="records"), mimetype='application/json')

# Executa o servidor Flask

In [ ]:
if __name__=="__main__":
    API.run(host=os.getenv('IP', '0.0.0.0'), 
            port=int(os.getenv('PORT', 4320)))

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:4320
 * Running on http://192.168.0.33:4320
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 21:51:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 21:51:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 21:52:08] "GET /Player_Teams HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 21:52:11] "GET /Player_Teams HTTP/1.1" 200 -
